In [96]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import math

In [97]:
# Load the data
df = pd.read_csv("01_dataset.csv", index_col="customer_id")
df.head()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,...,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
customer_id,,,,,,,,,,,,,,,,,,,,,
1639,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,2.000000,2.000000,2.000000,7.000000,3.666667,7.0,3.666667,3.666667,1000000.0
23717,0,10.0,1.0,NaN,7.0,1.0,1.0,4.0,NaN,NaN,...,2.000000,7.000000,2.000000,2.000000,7.000000,7.000000,7.0,7.000000,7.000000,NaN
19474,0,7.0,7.0,NaN,7.0,NaN,1.0,1.0,4.0,NaN,...,7.000000,5.571429,5.571429,5.571429,7.000000,7.000000,7.0,7.000000,7.000000,1000620.0
15505,0,7.0,1.0,1.0,NaN,1.0,NaN,7.0,1.0,1.0,...,6.090909,7.000000,NaN,NaN,4.500000,5.333333,7.0,5.333333,3.666667,1000170.0
1538,1,NaN,NaN,1.0,1.0,1.0,1.0,1.0,4.0,1.0,...,2.000000,2.000000,2.000000,2.000000,3.666667,7.000000,7.0,7.000000,3.666667,1000270.0


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1639 to 18772
Columns: 123 entries, label to OUTSTANDING_BAL_ALL_CURRENT
dtypes: float64(122), int64(1)
memory usage: 18.9 MB


In [99]:
df.describe()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,...,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
count,20000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,...,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,1.800000e+04
mean,0.182200,6.040667,3.316833,1.063333,3.177000,1.610500,1.061333,3.850500,2.730833,1.002333,...,1.524688,1.956620,1.531301,1.013238,6.458641,6.591671,6.691829,6.402545,6.006104,1.000943e+06
std,0.386019,3.620554,3.147751,0.446094,2.211548,1.547231,0.437320,2.550394,2.519339,0.089415,...,4.182101,4.304385,3.967109,3.560061,1.147119,1.022613,0.914260,1.177294,1.431080,4.007350e+03
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000e+06
25%,0.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-3.000000,-3.000000,-3.000000,-3.000000,7.000000,7.000000,7.000000,7.000000,4.500000,1.000100e+06
50%,0.000000,7.000000,1.000000,1.000000,4.000000,1.000000,1.000000,4.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000290e+06
75%,0.000000,7.000000,4.000000,1.000000,4.000000,1.000000,1.000000,4.000000,4.000000,1.000000,...,7.000000,7.000000,5.333333,3.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.000770e+06
max,1.000000,25.000000,25.000000,7.000000,13.000000,22.000000,7.000000,16.000000,19.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,1.169190e+06


In [100]:
# Check for missing values, find columns with missing values
null_count = df.isnull().sum()
null_cols = null_count[null_count>0].index.to_list()
print("The number of columns with missing values:", len(null_cols))

The number of columns with missing values: 122


=> Tất cả các cột ngoại trừ customer_id và label đều có giá trị null

In [101]:
df.notnull().all(axis=1).sum()

0

=> Tất cả các hàng đều có ít nhất một giá trị null => Không thể xóa hết các hàng có chứa giá trị rỗng được, có thể nghĩ đến hướng impute data

In [102]:
cols = df.columns
integer_cols = [col for col in cols if 'enquiries' in col.lower() or "count" in col.lower() or "number" in col.lower() or 'num' in col.lower() or 'month' in col.lower()]
integer_cols[:5]

['SHORT_TERM_COUNT',
 'MID_TERM_COUNT',
 'LONG_TERM_COUNT',
 'SHORT_TERM_COUNT_BANK',
 'MID_TERM_COUNT_BANK']

In [103]:
# Kiểm tra xem số lượt tra cứu có hợp lệ không (phải là số nguyên)

def check_invalid_enquiries(value):
    if pd.isna(value):
        return False
    else:
        if int(value) != value:
            return False 
        return True
    
# Tìm các index tương ứng với các dòng có số lượt tra cứu không hợp lệ

invalid_rows = []
for col in integer_cols:
    temp = df[col].apply(check_invalid_enquiries)
    invalid_rows.append(temp[temp == False].index.to_list())

In [104]:
df.loc[invalid_rows[-1]][integer_cols[-1]]

customer_id
1639     3.666667
15505    3.666667
1538     3.666667
20339    3.250000
6629     5.333333
           ...   
1620     4.500000
2022     4.500000
9301     4.500000
22422    4.500000
11823    5.333333
Name: ENQUIRIES_FROM_NON_BANK_3M_12M, Length: 7575, dtype: float64

In [105]:
def check_not_int(value):
    if pd.isna(value):
        return True
    return int(value) == value

for col in df.columns:
    if df[col].apply(check_not_int).sum() != df.shape[0]:
        print(col, df[col].apply(check_not_int).sum())

OUTSTANDING_BAL_LOAN_3M 9341
OUTSTANDING_BAL_LOAN_6M 9129
OUTSTANDING_BAL_LOAN_9M 5133
OUTSTANDING_BAL_LOAN_12M 6073
OUTSTANDING_BAL_CC_3M 13670
OUTSTANDING_BAL_CC_6M 13210
OUTSTANDING_BAL_CC_9M 10688
OUTSTANDING_BAL_CC_12M 11187
OUTSTANDING_BAL_ALL_3M 8993
OUTSTANDING_BAL_ALL_6M 8767
OUTSTANDING_BAL_ALL_9M 4818
OUTSTANDING_BAL_ALL_12M 5821
OUTSTANDING_BAL_LOAN_3M_6M 6474
OUTSTANDING_BAL_LOAN_6M_9M 7723
OUTSTANDING_BAL_LOAN_9M_12M 3289
OUTSTANDING_BAL_LOAN_6M_12M 7301
OUTSTANDING_BAL_LOAN_3M_12M 5569
OUTSTANDING_BAL_CC_3M_6M 12551
OUTSTANDING_BAL_CC_6M_9M 13122
OUTSTANDING_BAL_CC_9M_12M 9560
OUTSTANDING_BAL_CC_6M_12M 13060
OUTSTANDING_BAL_CC_3M_12M 11928
OUTSTANDING_BAL_ALL_3M_6M 5385
OUTSTANDING_BAL_ALL_6M_9M 6540
OUTSTANDING_BAL_ALL_9M_12M 3010
OUTSTANDING_BAL_ALL_6M_12M 6327
OUTSTANDING_BAL_ALL_3M_12M 4809
ENQUIRIES_3M_6M 15085
ENQUIRIES_6M_9M 15816
ENQUIRIES_9M_12M 16342
ENQUIRIES_6M_12M 14134
ENQUIRIES_3M_12M 12057
ENQUIRIES_FROM_BANK_3M_6M 18522
ENQUIRIES_FROM_BANK_6M_9M 18144
EN

In [106]:
# Xử lý các giá trị không hợp lệ bằng cách làm tròn giá trị về số nguyên gần nhất

def replace_float_to_int(value):
    if pd.isna(value):
        return value
    return math.floor(value) if math.ceil(value) - value > value - math.floor(value) else math.ceil(value)

for col in integer_cols:
    df[col] = df[col].apply(replace_float_to_int)

In [107]:
df.loc[invalid_rows[-1]][integer_cols[-1]]

customer_id
1639     4.0
15505    4.0
1538     4.0
20339    3.0
6629     5.0
        ... 
1620     5.0
2022     5.0
9301     5.0
22422    5.0
11823    5.0
Name: ENQUIRIES_FROM_NON_BANK_3M_12M, Length: 7575, dtype: float64

In [108]:
df.columns[40:]

Index(['OUTSTANDING_BAL_ALL_3M', 'OUTSTANDING_BAL_ALL_6M',
       'OUTSTANDING_BAL_ALL_9M', 'OUTSTANDING_BAL_ALL_12M',
       'OUTSTANDING_BAL_LOAN_3M_6M', 'OUTSTANDING_BAL_LOAN_6M_9M',
       'OUTSTANDING_BAL_LOAN_9M_12M', 'OUTSTANDING_BAL_LOAN_6M_12M',
       'OUTSTANDING_BAL_LOAN_3M_12M', 'OUTSTANDING_BAL_CC_3M_6M',
       'OUTSTANDING_BAL_CC_6M_9M', 'OUTSTANDING_BAL_CC_9M_12M',
       'OUTSTANDING_BAL_CC_6M_12M', 'OUTSTANDING_BAL_CC_3M_12M',
       'OUTSTANDING_BAL_ALL_3M_6M', 'OUTSTANDING_BAL_ALL_6M_9M',
       'OUTSTANDING_BAL_ALL_9M_12M', 'OUTSTANDING_BAL_ALL_6M_12M',
       'OUTSTANDING_BAL_ALL_3M_12M', 'INCREASING_BAL_3M_LOAN',
       'INCREASING_BAL_6M_LOAN', 'INCREASING_BAL_3M_CC',
       'INCREASING_BAL_6M_CC', 'INCREASING_BAL_3M_ALL',
       'INCREASING_BAL_6M_ALL', 'OUTSTANDING_BAL_CC_CURRENT',
       'CREDIT_CARD_MONTH_SINCE_10DPD', 'CREDIT_CARD_MONTH_SINCE_30DPD',
       'CREDIT_CARD_MONTH_SINCE_60DPD', 'CREDIT_CARD_MONTH_SINCE_90DPD',
       'CREDIT_CARD_NUMBER_OF_LATE

In [109]:
df['NUM_NEW_LOAN_TAKEN_3M'].apply(check_invalid_enquiries).sum()

18000

In [110]:
df[['OUTSTANDING_BAL_ALL_3M_6M', 'OUTSTANDING_BAL_ALL_3M', 'OUTSTANDING_BAL_ALL_6M']].head()

,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M
customer_id,,,
1639,1.000000e+06,1.000000e+06,1.000000e+06
23717,1.000014e+06,1.000433e+06,1.000320e+06
19474,1.000012e+06,1.000607e+06,1.000522e+06
15505,1.000010e+06,1.000097e+06,1.000098e+06
1538,1.000013e+06,NaN,1.000230e+06


In [111]:
df['OUTSTANDING_BAL_ALL_6M']-df['OUTSTANDING_BAL_ALL_3M']

customer_id
1639       0.000000
23717   -113.333333
19474    -85.000000
15505      1.666667
1538            NaN
            ...    
13215     -6.666667
938             NaN
11823     76.666667
14609           NaN
18772     18.333333
Length: 20000, dtype: float64

In [112]:
# Kiểm tra lại xem các cột số nguyên đã hợp lệ chưa
for col in df.columns:
    if df[col].apply(check_not_int).sum() != df.shape[0]:
        print(col, df[col].apply(check_not_int).sum())

OUTSTANDING_BAL_LOAN_3M 9341
OUTSTANDING_BAL_LOAN_6M 9129
OUTSTANDING_BAL_LOAN_9M 5133
OUTSTANDING_BAL_LOAN_12M 6073
OUTSTANDING_BAL_CC_3M 13670
OUTSTANDING_BAL_CC_6M 13210
OUTSTANDING_BAL_CC_9M 10688
OUTSTANDING_BAL_CC_12M 11187
OUTSTANDING_BAL_ALL_3M 8993
OUTSTANDING_BAL_ALL_6M 8767
OUTSTANDING_BAL_ALL_9M 4818
OUTSTANDING_BAL_ALL_12M 5821
OUTSTANDING_BAL_LOAN_3M_6M 6474
OUTSTANDING_BAL_LOAN_6M_9M 7723
OUTSTANDING_BAL_LOAN_9M_12M 3289
OUTSTANDING_BAL_LOAN_6M_12M 7301
OUTSTANDING_BAL_LOAN_3M_12M 5569
OUTSTANDING_BAL_CC_3M_6M 12551
OUTSTANDING_BAL_CC_6M_9M 13122
OUTSTANDING_BAL_CC_9M_12M 9560
OUTSTANDING_BAL_CC_6M_12M 13060
OUTSTANDING_BAL_CC_3M_12M 11928
OUTSTANDING_BAL_ALL_3M_6M 5385
OUTSTANDING_BAL_ALL_6M_9M 6540
OUTSTANDING_BAL_ALL_9M_12M 3010
OUTSTANDING_BAL_ALL_6M_12M 6327
OUTSTANDING_BAL_ALL_3M_12M 4809
